# Installs

In [ ]:
%%writefile requirements.txt

radiant-mlhub==0.4.1
rasterio
shapely
accelerate==0.15.0
albumentations==1.3.0
timm==0.6.12
watermark

Overwriting requirements.txt


In [ ]:
!pip install -q torch==1.13.1 torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

In [ ]:
!pip install -q -r requirements.txt

# Specs

In [ ]:
!nvidia-smi

Sun Mar 12 12:25:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:05.0 Off |                    0 |
| N/A   29C    P0    49W / 400W |      0MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Imports

In [ ]:
import os, gc
import copy
import math
import random
import getpass
import pandas as pd
import numpy as np
from time import time, sleep
from tqdm.notebook import tqdm
from collections import Counter, defaultdict
from scipy.ndimage import gaussian_filter

from glob import glob
import warnings

import shutil

import subprocess
from joblib import Parallel, delayed
import multiprocessing

import cv2
import PIL
from PIL import Image
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import f1_score, mean_squared_error

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset as TDataset, DataLoader
from torch.optim.lr_scheduler import LambdaLR, CosineAnnealingLR

from accelerate import Accelerator

import albumentations as A
import albumentations.augmentations.geometric.transforms as AG

import timm

In [ ]:
%load_ext watermark
%watermark --iversions

pandas        : 1.5.0
PIL           : 9.2.0
numpy         : 1.23.4
torch         : 1.13.1+cu116
albumentations: 1.3.0
matplotlib    : 3.6.1
cv2           : 4.7.0
timm          : 0.6.12



# Envs

In [ ]:
def disable_warnings(strict=False):
	warnings.simplefilter('ignore')
	if strict:
		logging.disable(logging.WARNING)

def seed_everything(seed=42):
	random.seed(seed)
	os.environ['PYTHONHASHSEED'] = str(seed)
	np.random.seed(seed)
	torch.manual_seed(seed)
	torch.cuda.manual_seed(seed)
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn.benchmark = False

In [ ]:
SEED = 42

disable_warnings()
seed_everything(SEED)

# Data

In [ ]:
# load files
data_path = './data/'

train = pd.read_csv(data_path + 'Train.csv')
test = pd.read_csv(data_path + 'Test.csv')
sample_submission = pd.read_csv(data_path + 'SampleSubmission.csv')

# Preview train
train.head()

,ImageId,Target
0,Id_jdqw9hlv6j.png,14.0
1,Id_6xtrolmuvc.png,18.0
2,Id_2m49sj3xd9.png,0.0
3,Id_9jwg5pcnn4.png,28.0
4,Id_vnm6e8n0p3.png,21.0


In [ ]:
# Preview test
test.head()

,ImageId
0,Id_ohk78h9ld8.png
1,Id_eeyj2u4j7y.png
2,Id_wsd7vx2ifa.png
3,Id_6vfneamaoh.png
4,Id_9wil3575fv.png


In [ ]:
# Preview sample submission
sample_submission.head()

,ImageId,Target
0,Id_ohk78h9ld8.png,0
1,Id_eeyj2u4j7y.png,0
2,Id_wsd7vx2ifa.png,0
3,Id_6vfneamaoh.png,0
4,Id_9wil3575fv.png,0


# Utils

In [ ]:
def paralellize(fct, data, verbose=0, with_tqdm=True):
    fn = map(delayed(fct), data)
    if with_tqdm:
        fn = tqdm(fn, total=len(data))
    return Parallel(n_jobs=-1, verbose=verbose, backend="multiprocessing")(fn)

In [ ]:
def get_common_transforms(size):
    return A.Compose([
      A.Resize(height=size, width=size, p=1),
    ])

def get_train_transforms():
    return A.Compose([
        A.Flip(),
        A.RandomRotate90(),
        A.OneOf([
            A.MotionBlur(),
            A.MedianBlur(blur_limit=3),
            A.Blur(blur_limit=3),
        ]),
    ])

def get_mosaic_transforms():
    return A.Compose([
        A.Flip(),
        A.RandomRotate90(),
        A.ShiftScaleRotate(
            shift_limit=0.0625, scale_limit=(-0.3, -0.2), rotate_limit=45, interpolation=1, border_mode=0, value=0, p=0.2
        )
    ])

In [ ]:
def get_concat_h(im1, im2):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

def get_concat_v(im1, im2):
    dst = Image.new('RGB', (im1.width, im1.height + im2.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst

def get_combos(ids, n=4) :
    combos = []
    size = len(ids)
    
    for i in range(0, size - n, 1):
        combos.append(ids[i:i+n])
    return combos

def _combo_image_saver(ids_list):
    save_path = './data/TreeImagesCombo/'
    
    images = [
        Image.open(f'./data/TreeImages/{p}') for p in ids_list
    ]

    img = get_concat_h(*images[:2])

    if len(ids_list) == 4:
        img = get_concat_v(
            img,
            get_concat_h(*images[2:])
        )

    ids_list = [p.split('.')[0] for p in ids_list]
    path = save_path + '|'.join(ids_list) + '.png'

    img.save(path)

def concat_mix_up(zeros_ids, palm_ids, n=4):
    np.random.shuffle(zeros_ids)
    np.random.shuffle(palm_ids)
    
    palm_combos = []
    if n != -1:
        palm_combos = get_combos(palm_ids, n=n)
    
    new_palm_combos = list(zip(zeros_ids, palm_ids))
    new_palm_combos = [list(x) for x in new_palm_combos]
    
    _ = paralellize(_combo_image_saver, palm_combos + new_palm_combos)
    
    print(f'New {len(palm_combos + new_palm_combos)} images generated')
    
    return palm_combos + new_palm_combos

In [ ]:
def mosaic(imgs):
    size = len(imgs)
    h, w, c = imgs[0].shape
    
    assert 1 <= size <= 4, f'size is {size}'
    
    if size == 1:
        img = imgs[0]
    elif size == 2:
        img = np.empty(shape=(h, w*size, c), dtype=imgs[0].dtype)
        img[:, :w, :] = imgs[0]
        img[:, w:, :] = imgs[1]
    else:
        if size == 3:
            imgs.append(np.zeros_like(imgs[0], dtype=imgs[0].dtype))
        
        img = np.empty(shape=(h*2, w*2, c), dtype=imgs[0].dtype)
        img[:h, :w, :] = imgs[0]
        img[h:, :w, :] = imgs[1]
        img[:h, w:, :] = imgs[2]
        img[h:, w:, :] = imgs[3]
    
    return img

In [ ]:
def criterion(preds, biomass):
    loss_fn = nn.MSELoss()
    return torch.sqrt(loss_fn(preds, biomass))

def metric(preds, biomass):
    preds = preds.cpu().numpy()
    preds = np.where(preds > 0, preds, 0)
    
    return mean_squared_error(biomass.cpu().numpy(), preds, squared=False)

In [ ]:
class TreeCountingModel(nn.Module):
    def __init__(self, args):
        super().__init__()

        self.backbone = timm.create_model(
            args.arch, num_classes=1, pretrained=True, **args.extra_params
        )

    def forward(self, x):
        x = self.backbone(x)
        return x

In [ ]:
class TreeCountingDataset(torch.utils.data.Dataset):
    def __init__(self, args, imagesPath, df, tfms=None, phase='test'):
        self.imagespath = imagesPath
        self.df = df.reset_index(drop=True)
        self.tfms = tfms
        self.default_tfms = get_common_transforms(args.img_size)
        self.mosaic = args.mosaic
        self.mosaic_aug = args.mosaic_aug
        self.mosaic_tfms = get_mosaic_transforms()
        self.length = len(self.df)
        self.phase = phase

    def __len__(self):
        return self.length
    
    def _read_image(self, idx):
        imagename, label, *z = self.df.iloc[idx].values
        path = os.path.join(self.imagespath, imagename)
        if not os.path.exists(path):
            path = os.path.join('./data/TreeImagesCombo/', imagename)
        image = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
        
        return image, label

    def __getitem__(self, idx):
        image, label = self._read_image(idx)
        
        if self.mosaic and self.phase == 'train':
            n_image = np.random.choice([0, 1])
            if n_image != 0:
                idxs = np.random.randint(self.length, size=n_image)
                images = [image]
                for i in idxs:
                    img, i_label = self._read_image(i)
                    if self.mosaic_aug:
                        img = self.mosaic_tfms(image=img)['image']
                        
                    images.append(img)
                    label += i_label
                
                image = mosaic(images)
        
        if self.tfms is not None:
            image = self.tfms(image=image)['image']
        
        image = self.default_tfms(image=image)['image']
        image = torch.tensor(image, dtype=torch.float)
        image = torch.permute(image, (2, 0, 1))
        
        return image, torch.tensor([label], dtype=torch.float)

In [ ]:
def get_model(args):
    model = TreeCountingModel(args)

    return model

In [ ]:
def make_splits(args, df, path='folds/'):
    os.makedirs("folds/", exist_ok=True)
    os.makedirs(f"folds/{args.n_split}_splits/", exist_ok=True)
    
    all_splits = {}
    fold_path = f"folds/{args.n_split}_splits/"

    df_clean = df.copy()
    
    to_drop = ['Id_u45dpub99b.png', 'Id_kzuy1er5jm.png', 'Id_lp4yl8q9n2.png', 'Id_2r2259ynzp.png', 'Id_w4fnd54go8.png', 'Id_lh8b1k1lx8.png', 'Id_hmcnf15rll.png']
    hard_ids = ['Id_scf1bqac43.png', 'Id_uhcehdmyc1.png', 'Id_y87obok3v0.png', 'Id_kx60b4dgn2.png', 'Id_85992p4se2.png', 'Id_g7fbxrng46.png',
           'Id_tz74cy4nh9.png', 'Id_u2dzh57wmf.png', 'Id_n0ipc9o8el.png', 'Id_lvua92vvg5.png', 'Id_ibrflyue8t.png', 'Id_xi61ipimpj.png',
           'Id_tn748l5k57.png', 'Id_63zqtghlbi.png', 'Id_9xwykwajjb.png', 'Id_owxhn1zmlh.png', 'Id_6ls94ewz47.png', 'Id_upua3g7q2q.png',
           'Id_eza4ic1a4k.png', 'Id_jxu8riswpw.png', 'Id_4nuo9gc80n.png', 'Id_rkm6vwm0md.png', 'Id_2goic5xoft.png', 'Id_234nhwag1a.png',
           'Id_842atkbl2g.png', 'Id_v9gmdly1tl.png', 'Id_7evqslin0q.png', 'Id_5jgknuixmg.png', 'Id_8bhp0orrri.png', 'Id_yyvlvcursz.png',
           'Id_kbgnlekbjm.png', 'Id_kd9m5eoeuf.png', 'Id_h0f091di0y.png', 'Id_scf4jxhlfg.png', 'Id_8xuqs2ut71.png', 'Id_4x0zh3y93q.png',
           'Id_oamy4tybj9.png', 'Id_8q73x3g38y.png', 'Id_k0c89tgg4c.png', 'Id_niwpztq309.png', 'Id_r405nptkvd.png'
    ]
    df_hard = df_clean[df_clean.ImageId.isin(hard_ids)].reset_index(drop=True)
    df_clean = df_clean[~df_clean.ImageId.isin(to_drop+hard_ids)].reset_index(drop=True)
    
    kf = StratifiedKFold(args.n_split, shuffle=True, random_state=42)
    
    num_bins = int(np.floor(1 + np.log2(len(df_clean))))
    bins = pd.cut(df_clean['Target'], bins=num_bins, labels=False)
        
    print('Num bins:', num_bins)
    
    for f, (tr_idx, vr_idx) in enumerate(kf.split(df_clean, bins)):
        all_splits[f] = {
          'train': pd.concat([ df_clean.iloc[tr_idx], df_hard]).sample(frac=1),
          'val': df_clean.iloc[vr_idx],
        }
        
        df_clean.iloc[vr_idx].to_csv(f"folds/{args.n_split}_splits/fold_{f}.csv", index=False)

    return all_splits

In [ ]:
class Timer:
    def __init__(self):
        self._time = 0

    def start(self):
        self._time = time()

    @property
    def time(self):
        return (time() - self._time) / 60

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
def training_fn(args, dataloader, model, opt, epoch, accelerator, scheduler=None):    
    avg_loss = 0

    size = len(dataloader)
    timer = Timer()
    meter = AverageMeter()

    model.train()
    timer.start()
    for i, (x, biomass) in enumerate(dataloader):
        preds = model(x)
        loss = criterion(preds, biomass)

        avg_loss += loss.item()

        accelerator.backward(loss)
        opt.step()
        opt.zero_grad()

        score = metric(preds.detach().float(), biomass.detach())
        meter.update(score)

        if scheduler is not None and args.schedule not in ['plateau']:
            scheduler.step()

        print(('\r[Training][{}/{}][{:.2f} min] Epoch {} : Loss: {:.5f} - RMSE: {:.5f}').format(
        i+1, size, timer.time, epoch, avg_loss/(i+1), meter.avg
        ), end='')
    print()

################## evaluation Function ####################   

def evaluate(dataloader, model):
    avg_loss = 0

    all_logits = []
    all_masks = []

    size = len(dataloader)
    timer = Timer()
    meter = AverageMeter()

    model.eval()
    timer.start()
    with torch.no_grad():
        for i, (x, biomass) in enumerate(dataloader):
            preds = model(x)
            loss = criterion(preds, biomass)

            avg_loss += loss.item()

            score = metric(preds.detach().float(), biomass.detach().float())
            meter.update(score)

            print(('\r[Evaluation][{}/{}][{:.2f} min] Loss: {:.5f} - RMSE: {:.5f}').format(
            i+1, size, timer.time, avg_loss/(i+1), meter.avg
            ), end='')
    print()

    return avg_loss/size, meter.avg

################## Run training over folds Function ####################   
def release_memory():
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.reset_max_memory_allocated()
    torch.cuda.synchronize()

def run_fold(fold, args, path='./models'):
    model_path = f'{path}/{args.arch}-{args.n_split}-{args.img_size}-{args.lr}-{args.bs}'
    if fold == 0:
        mid = len(glob(model_path+'*'))
        model_path += f'-{mid}'
        os.makedirs(model_path, exist_ok=False)
        
        print('Model path:', model_path)
    else:
        model_id = sorted([int(x.replace(model_path+'-', '')) for x in glob(f'{model_path}*')])[-1]
        model_path = f'{model_path}-{model_id}'
    
    best_metric = np.inf
    best_loss = np.inf
    
    train_ds = TreeCountingDataset(
        args,
        imagesPath='./data/TreeImages/',
        df=all_splits[fold]['train'],
        tfms=get_train_transforms(),
        phase='train'
    )
    val_ds = TreeCountingDataset(
        args,
        imagesPath='./data/TreeImages/',
        df=all_splits[fold]['val'],
    )

    trainloader = DataLoader(train_ds, batch_size=args.bs, num_workers=args.workers, pin_memory=True, shuffle=True)
    validloader = DataLoader(val_ds, batch_size=args.ebs, num_workers=args.workers, pin_memory=True, shuffle=False)

    accelerator = Accelerator(fp16=args.fp16)

    model = get_model(args)
    opt = torch.optim.AdamW(model.parameters(), lr=args.lr, weight_decay=args.wd)

    num_train_steps = int(len(trainloader) * args.epochs)
    steps_per_epoch = len(trainloader)

    scheduler = None
    if args.schedule:
        scheduler = CosineAnnealingLR(opt, T_max=num_train_steps, eta_min=args.min_lr)
        model, scheduler, opt, trainloader, validloader = accelerator.prepare(model, scheduler, opt, trainloader, validloader)
    else:
        model, opt, trainloader, validloader = accelerator.prepare(model, opt, trainloader, validloader)

    os.makedirs(path, exist_ok=True)

    loader = tqdm(range(args.epochs), desc=f'Fold {fold}')

    for epoch in loader:
        training_fn(args, trainloader, model, opt, epoch, accelerator, scheduler)
        avg_loss, avg_metric = evaluate(validloader, model)

        if scheduler is not None and args.schedule in ['plateau']:
            scheduler.step(avg_loss)

        if avg_metric < best_metric:
            best_loss = avg_loss
            best_metric = avg_metric

            torch.save(model.state_dict(), f'{model_path}/best_{fold}.pt')
        print()


    print(f"Best Fold --- Loss score: {best_loss} - F1 Score {best_metric}")

    # Clean up
    del trainloader, validloader
    del model, opt
    release_memory()

    return best_metric

# Training

In [ ]:
class CFG:
    arch = 'convnext_small_384_in22ft1k'
    lr = 1e-4
    min_lr = 0
    wd = 1e-6
    epochs = 20
    warmup = 0.
    bs = 8
    ebs = 2 * bs
    
    n_split = 5
    use_folds = list(range(n_split))
    actual_split = len(use_folds)

    img_size = 1024
    
    tta_tfms = []

    schedule = False
    augment = True
    mosaic = False
    mosaic_aug = False

    fp16 = True
    workers = os.cpu_count()
    
    print('Cpu count:', workers)

    extra_params = {
    }

Cpu count: 12


In [ ]:
args = CFG()

In [ ]:
all_splits = make_splits(args, train)

Num bins: 11


In [ ]:
avg_score = 0

for f in args.use_folds:
    fold_score = run_fold(f, args)
    avg_score += (fold_score / args.actual_split)
    
    print()

Model path: ./models/convnext_small_384_in22ft1k-5-1024-0.0001-8-0


Fold 0:   0%|          | 0/20 [00:00<?, ?it/s]

[Training][201/201][1.51 min] Epoch 0 : Loss: 7.95071 - RMSE: 7.9445584
[Evaluation][25/25][0.15 min] Loss: 4.20380 - RMSE: 4.19822

[Training][201/201][1.46 min] Epoch 1 : Loss: 3.18403 - RMSE: 3.16384
[Evaluation][25/25][0.10 min] Loss: 2.86844 - RMSE: 2.83813

[Training][201/201][1.46 min] Epoch 2 : Loss: 2.89364 - RMSE: 2.87608
[Evaluation][25/25][0.11 min] Loss: 2.69747 - RMSE: 2.69747

[Training][201/201][1.45 min] Epoch 3 : Loss: 2.43221 - RMSE: 2.41383
[Evaluation][25/25][0.11 min] Loss: 2.37701 - RMSE: 2.37701

[Training][201/201][1.46 min] Epoch 4 : Loss: 2.03921 - RMSE: 2.02404
[Evaluation][25/25][0.10 min] Loss: 2.07280 - RMSE: 2.07280

[Training][201/201][1.46 min] Epoch 5 : Loss: 2.00674 - RMSE: 1.99339
[Evaluation][25/25][0.11 min] Loss: 1.90494 - RMSE: 1.90494

[Training][201/201][1.46 min] Epoch 6 : Loss: 1.95307 - RMSE: 1.94440
[Evaluation][25/25][0.11 min] Loss: 1.68458 - RMSE: 1.68458

[Training][201/201][1.46 min] Epoch 7 : Loss: 1.78953 - RMSE: 1.78476
[Evaluation

Fold 1:   0%|          | 0/20 [00:00<?, ?it/s]

[Training][201/201][1.46 min] Epoch 0 : Loss: 7.11201 - RMSE: 7.109146
[Evaluation][25/25][0.10 min] Loss: 3.72139 - RMSE: 3.69417

[Training][201/201][1.46 min] Epoch 1 : Loss: 3.17880 - RMSE: 3.16478
[Evaluation][25/25][0.10 min] Loss: 2.14418 - RMSE: 2.14415

[Training][201/201][1.47 min] Epoch 2 : Loss: 2.61273 - RMSE: 2.60141
[Evaluation][25/25][0.11 min] Loss: 1.97604 - RMSE: 1.96725

[Training][201/201][1.46 min] Epoch 3 : Loss: 2.20010 - RMSE: 2.18757
[Evaluation][25/25][0.10 min] Loss: 1.84944 - RMSE: 1.84110

[Training][201/201][1.47 min] Epoch 4 : Loss: 2.07859 - RMSE: 2.06833
[Evaluation][25/25][0.11 min] Loss: 1.89189 - RMSE: 1.89189

[Training][201/201][1.46 min] Epoch 5 : Loss: 1.88416 - RMSE: 1.87772
[Evaluation][25/25][0.11 min] Loss: 1.61339 - RMSE: 1.61116

[Training][201/201][1.46 min] Epoch 6 : Loss: 1.99709 - RMSE: 1.98805
[Evaluation][25/25][0.10 min] Loss: 1.57929 - RMSE: 1.57929

[Training][201/201][1.46 min] Epoch 7 : Loss: 1.79836 - RMSE: 1.79028
[Evaluation]

...

[Training][201/201][1.46 min] Epoch 0 : Loss: 8.67056 - RMSE: 8.6676135
[Evaluation][25/25][0.11 min] Loss: 4.65368 - RMSE: 4.64621

[Training][201/201][1.46 min] Epoch 1 : Loss: 3.28305 - RMSE: 3.27810
[Evaluation][25/25][0.10 min] Loss: 2.34180 - RMSE: 2.34179

[Training][201/201][1.46 min] Epoch 2 : Loss: 2.46573 - RMSE: 2.45603
[Evaluation][25/25][0.10 min] Loss: 2.34573 - RMSE: 2.34573

[Training][201/201][1.46 min] Epoch 3 : Loss: 2.09260 - RMSE: 2.08336
[Evaluation][25/25][0.10 min] Loss: 1.74032 - RMSE: 1.73995

[Training][201/201][1.47 min] Epoch 4 : Loss: 1.97146 - RMSE: 1.96553
[Evaluation][25/25][0.10 min] Loss: 2.81178 - RMSE: 2.81178

[Training][201/201][1.46 min] Epoch 5 : Loss: 1.97935 - RMSE: 1.97346
[Evaluation][25/25][0.11 min] Loss: 2.19962 - RMSE: 2.19962

[Training][201/201][1.47 min] Epoch 6 : Loss: 1.81429 - RMSE: 1.80574
[Evaluation][25/25][0.11 min] Loss: 1.93236 - RMSE: 1.93156

[Training][201/201][1.46 min] Epoch 7 : Loss: 1.80018 - RMSE: 1.79478
[Evaluation

Fold 3:   0%|          | 0/20 [00:00<?, ?it/s]

[Training][201/201][1.47 min] Epoch 0 : Loss: 11.23300 - RMSE: 11.23137
[Evaluation][25/25][0.11 min] Loss: 7.91402 - RMSE: 7.91402

[Training][201/201][1.46 min] Epoch 1 : Loss: 5.12688 - RMSE: 5.12051
[Evaluation][25/25][0.11 min] Loss: 3.62487 - RMSE: 3.59348

[Training][201/201][1.47 min] Epoch 2 : Loss: 2.69840 - RMSE: 2.68637
[Evaluation][25/25][0.11 min] Loss: 2.16339 - RMSE: 2.15486

[Training][201/201][1.46 min] Epoch 3 : Loss: 2.33528 - RMSE: 2.32594
[Evaluation][25/25][0.11 min] Loss: 2.13405 - RMSE: 2.08305

[Training][201/201][1.46 min] Epoch 4 : Loss: 2.22750 - RMSE: 2.21284
[Evaluation][25/25][0.11 min] Loss: 1.81662 - RMSE: 1.80549

[Training][201/201][1.46 min] Epoch 5 : Loss: 1.93024 - RMSE: 1.92347
[Evaluation][25/25][0.11 min] Loss: 1.72613 - RMSE: 1.72537

[Training][201/201][1.46 min] Epoch 6 : Loss: 1.96618 - RMSE: 1.95685
[Evaluation][25/25][0.11 min] Loss: 1.56582 - RMSE: 1.56465

[Training][201/201][1.47 min] Epoch 7 : Loss: 1.77780 - RMSE: 1.76975
[Evaluation

Fold 4:   0%|          | 0/20 [00:00<?, ?it/s]

[Training][201/201][1.47 min] Epoch 0 : Loss: 6.76669 - RMSE: 6.761642
[Evaluation][25/25][0.10 min] Loss: 4.09242 - RMSE: 4.09242

[Training][201/201][1.47 min] Epoch 1 : Loss: 3.10128 - RMSE: 3.09028
[Evaluation][25/25][0.11 min] Loss: 2.48499 - RMSE: 2.48499

[Training][201/201][1.46 min] Epoch 2 : Loss: 2.49536 - RMSE: 2.48652
[Evaluation][25/25][0.11 min] Loss: 2.17969 - RMSE: 2.15183

[Training][201/201][1.47 min] Epoch 3 : Loss: 2.13085 - RMSE: 2.12045
[Evaluation][25/25][0.11 min] Loss: 2.30432 - RMSE: 2.30432

[Training][201/201][1.46 min] Epoch 4 : Loss: 2.03298 - RMSE: 2.02247
[Evaluation][25/25][0.10 min] Loss: 2.33213 - RMSE: 2.31242

[Training][201/201][1.46 min] Epoch 5 : Loss: 1.94478 - RMSE: 1.93691
[Evaluation][25/25][0.10 min] Loss: 1.61599 - RMSE: 1.61599

[Training][201/201][1.47 min] Epoch 6 : Loss: 1.83508 - RMSE: 1.82967
[Evaluation][25/25][0.11 min] Loss: 1.60943 - RMSE: 1.60942

[Training][201/201][1.47 min] Epoch 7 : Loss: 1.75933 - RMSE: 1.75376
[Evaluation]

In [ ]:
avg_score

1.3955025820732117